## SCOM Calculation from Logs

In [8]:
import json

#file = open("test_data/scenario1.json")
#file = open("test_data/scenario2.json")
#file = open("test_data/scenario3.json")
#file = open("test_data/insert.json")
#file = open("test_data/delete.json")
file = open("test_data/update.json")
result = json.load(file)
file.close()

result

{'data': [{'traceID': 'dd6aadda773b82b2fe199acd5e819f99',
   'spans': [{'traceID': 'dd6aadda773b82b2fe199acd5e819f99',
     'spanID': 'dde37d1222d1d1d4',
     'operationName': 'UPDATE',
     'references': [{'refType': 'CHILD_OF',
       'traceID': 'dd6aadda773b82b2fe199acd5e819f99',
       'spanID': '38054b21debf53b2',
       'span': {'traceID': 'dd6aadda773b82b2fe199acd5e819f99',
        'spanID': '38054b21debf53b2',
        'operationName': '/employees/updatePosition',
        'references': [],
        'startTime': 1716798542624959,
        'duration': 54141,
        'tags': [{'key': 'http.flavor', 'type': 'string', 'value': '1.1'},
         {'key': 'http.host', 'type': 'string', 'value': 'localhost:5001'},
         {'key': 'http.method', 'type': 'string', 'value': 'POST'},
         {'key': 'http.route',
          'type': 'string',
          'value': '/employees/updatePosition'},
         {'key': 'http.scheme', 'type': 'string', 'value': 'http'},
         {'key': 'http.server_name', 

### Extract table names

In [23]:
import re

table_name_pattern = re.compile(
    r"""
    (?i)   # Case-insensitive matching
    \bFROM\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)|   
    \bJOIN\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)|   
    \bINTO\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)|  
    \bUPDATE\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)| 
    \bDELETE\s+FROM\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)| 
    \bCREATE\s+TABLE\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)|
    \bDROP\s+TABLE\s+([`'"]?[a-zA-Z_][\w$]*[`'"]?)    
    """,
    re.VERBOSE
)

def extract_table_names(sql):
    matches = table_name_pattern.findall(sql)
    #matches = [
    #('employees', '', '', '', '', '', ''),
    #('', 'customers', '', '', '', '', '')]
    # filters out empty matches ('') and flattens result to normal list
    return [match for sublist in matches for match in sublist if match]

sql_statements = [
    "SELECT name, email FROM employees;",
    "SELECT * FROM employees e JOIN customers c ON e.id = c.employee_id;",
    "INSERT INTO orders (customer_id, product_id, order_date) VALUES (1, 2, '2024-05-25');",
    "UPDATE employees SET position = 'Manager' WHERE id = 1;",
    "DELETE FROM customers WHERE id = 1;",
    "CREATE TABLE products (id SERIAL PRIMARY KEY, name VARCHAR(100));",
    "DROP TABLE orders;"
]

for sql in sql_statements:
    print(f"SQL: {sql}")
    print(f"Tables: {extract_table_names(sql)}\n")


SQL: SELECT name, email FROM employees;
Tables: ['employees']

SQL: SELECT * FROM employees e JOIN customers c ON e.id = c.employee_id;
Tables: ['employees', 'customers']

SQL: INSERT INTO orders (customer_id, product_id, order_date) VALUES (1, 2, '2024-05-25');
Tables: ['orders']

SQL: UPDATE employees SET position = 'Manager' WHERE id = 1;
Tables: ['employees']

SQL: DELETE FROM customers WHERE id = 1;
Tables: ['customers']

SQL: CREATE TABLE products (id SERIAL PRIMARY KEY, name VARCHAR(100));
Tables: ['products']

SQL: DROP TABLE orders;
Tables: ['orders']



### Parse JSON Input

In [39]:
class Log:
    def __init__(self, span_id, spans, tags):
        self.span_id = span_id
        self.spans = spans
        self.tags = tags

    def __repr__(self):
        return f"Log(span_id={self.span_id}, spans={self.spans}, tags={self.tags})"

    def to_json(self):
        return json.dumps({
            'spanId': self.span_id,
            'spans': self.spans,
            'tags': self.tags
        }, indent=2)
    
    def get_operation_name(self):
        result = []
        for s in self.spans:
            result.append(s["operationName"])

        if len(result) > 0:
            input = result[0].split("/")
            return input[1]
        
        return None
    
    def get_db_statement(self):
        result = []
        for s in self.tags:
            if s["key"] == "db.statement":
                result.append(s["value"])

        if len(result) > 0:
            return result
        
        return None
    
    def get_table_names(self):
        statement = self.get_db_statement()
        if statement is not None:
            return extract_table_names(statement[0])

In [40]:
logs = []
for data in result["data"]:
    for log in data["spans"]:
        span_id = log['spanID']
        spans = []
        for r in log['references']: 
            spans.append(r["span"])

        tags = log['tags']
        span_obj = Log(span_id=span_id, spans=spans, tags=tags)
        logs.append(span_obj)

# Print all Span objects
for log in logs:
    print(log.span_id)
    print(log.get_operation_name())
    print(log.get_db_statement())
    print(log.get_table_names())

dde37d1222d1d1d4
employees
['UPDATE employees SET position = %s WHERE id = %s']
['employees']
38054b21debf53b2
None
None
None


In [46]:
grouped_logs= {}

for log in logs:
    operationName = log.get_operation_name()

    if operationName == None:
        continue

    if operationName not in grouped_logs:
        grouped_logs[operationName] = []

    table_name = log.get_table_names()

    if table_name in grouped_logs[operationName]: 
        continue
    
    for name in log.get_table_names():
        grouped_logs[operationName].append(name)

for operation, group in grouped_logs.items():
    print(f"Operation: {operation}")
    for log in group:
        print(f"  {log}")

print(grouped_logs)

Operation: employees
  employees
{'employees': ['employees']}


### Calculate SCOM

In [47]:
def calculateConnectionIntensity(i, j):
     common_attributes = set(i).intersection(j)
     if len(common_attributes) == 0: return 0

     return len(common_attributes) / (min(len(set(i)), len(set(j))))

def scom(apis, number_of_tables):
    print(f"Tabellenanzahl: {number_of_tables}")

    n_of_apis = len(apis)
    if n_of_apis <= 1: return "Too few endpoints"

    total_weighted_connections = 0

    processed_pairs = set()  # Verarbeitete Paare speichern

    for i, api1 in enumerate(apis):
        for api2 in list(apis.keys())[i + 1:]:
            pair_key = tuple(sorted((api1, api2)))
            
            if pair_key in processed_pairs:
                continue  # Überspringen, wenn Paar schon verarbeitet wurde
            
            connection_intensity = calculateConnectionIntensity(apis[api1], apis[api2])
            n_involved_tables = len(set(apis[api1]).union(set(apis[api2])))
            weight = n_involved_tables / number_of_tables
            weighted_connection = connection_intensity * weight
            total_weighted_connections += weighted_connection
            processed_pairs.add(pair_key)  # Paar als verarbeitet markieren

    return total_weighted_connections / (n_of_apis*(n_of_apis-1) / 2)

In [48]:
print(f"SCOM for {grouped_logs}: {scom(grouped_logs, 4)}")

Tabellenanzahl: 4
SCOM for {'employees': ['employees']}: Too few endpoints
